In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Importando las librerías estándares de Qiskit
from qiskit import QuantumCircuit, QuantumRegister, transpile, Aer, IBMQ, execute
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.aer import QasmSimulator
from tqdm.notebook import tqdm

from qiskit.providers.aer import QasmSimulator
from qiskit.tools.monitor import job_monitor
from qiskit.circuit import Parameter
import qiskit.quantum_info as qi

from qc_grader.challenges.spring_2022.helpers import generate_disordered_tb_instruction

# Suprime las advertencias
import warnings
warnings.filterwarnings('ignore')

## Dinámica cuántica de muchos cuerpos

En este ejercicio, examinamos el desorden en la red cristalina y las interacciones partícula-partícula. En un sistema cuántico de muchos cuerpos cerrado y preparado en un estado fuera del equilibrio, tenderá hacia el estado de equilibrio, lo que se conoce como termalización, siguiendo su propia dinámica. Este comportamiento es el resultado de las leyes de la mecánica estadística, y es análogo a una taza caliente con café que se enfría hasta alcanzar la temperatura del ambiente si se deja desatendida.

Sin embargo, la presencia del desorden en la red cristalina impide que el sistema evolucione hacia un estado ergódico termalizado. Esta acción conjunta entre el desorden y la interacción entre partículas resulta en una localización de muchos cuerpos, y permite que el sistema preserve la memoria de las condiciones iniciales para todo tiempo. Para más información, ver referencia [1]

Consideremos de nuevo un hamiltoniano de enlaces fuertes con energías por sitio arbitrarias:


$$H_{\rm tb}/\hbar = \sum^{3}_{i=0} (X_i X_{i+1} + Y_i Y_{i+1}) + \sum_i \epsilon_i Z_i $$

Como ya exploramos en el ejercicio 2, podemos emular el desorden en este sistema si escogemos $\epsilon_i=W \rm{cos}(2\pi\beta i)$, donde $W$ es la fuerza del desorden, y $\beta$ determina la periodicidad del cuasicristal. Podemos añadir varias partículas en el sistema y preparar múltiples qubits en el estado $|1\rangle$ antes de aplicar la evolución unitaria en el tiempo. Para $H_{\rm tb}$, cada sitio puede ser ocupado por una sola partícula, resultando en una interacción de repulsión entre partículas.


[1] https://arxiv.org/abs/1804.11065

Una de los signos del pérdida de la termalización es la emergencia de **desbalance**. En un sistema termalizado, esperamos que cada sitio de la red esté ocupado con el mismo número promedio de partículas después de alcanzar el estado estacionario. Sin embargo, añadir más desorden a un sistema en interacción resultaría en desviaciones del valor térmico observado en cada sitio. El desbalance del sistema se puede medir cuantitativamente usando la expresión siguiente:


$$\mathcal{I}= \langle (N_p-N_i)/(N_p+N_i) \rangle$$

donde $N_p$ y $N_i$ son los poblaciones pares e impares del sistema. Para un estado termalizado $\mathcal{I}=0$, y el desorden del sistema causará desviaciones de este valor.

Consideremos algunos ejemplos:

1. $ |\psi\rangle = |0111\rangle $: 
\begin{align*}
\langle \psi | N_p | \psi \rangle &= 1 \\
\langle \psi | N_i | \psi \rangle &= 2 \\
\mathcal{I} = \langle \psi |  (N_p-N_i)/(N_p+N_i) | \psi \rangle &= -1/3
\end{align*}

2. $ |\psi\rangle = \sqrt{2/3} |0111\rangle + \sqrt{1/3} |1011\rangle $: 
$$ \langle \psi |  \mathcal{I} | \psi \rangle = 2/3 * \langle 0111 |  \mathcal{I} |0111\rangle + 1/3 * \langle 1011 |  \mathcal{I} |1011\rangle $$
\begin{align*}
\langle 0111 |  \mathcal{I} |0111\rangle &= -1/3 \\
\langle 1011 |  \mathcal{I} |1011\rangle &= 1/3 \\
 \langle \psi |  \mathcal{I} | \psi \rangle &= -1/9
\end{align*}

<div class="alert alert-block alert-danger">
    
<b>Desafío, pregunta 3a</b> 

Escribe una función que regrese el desbalance un estado cuántico
    
</div>

In [ ]:
def get_imbalance(state):
    ###EDITA EL CÓDIGO DEBAJO
    ###PISTA:ASEGÚRATE DE OMITIR EL CÁLCULO DEL DESBALANCE DEL ESTADO |00...0>
    imbalance_val=0
    
    
    ###NO MODIFICAR DEBAJO DE ESTA LÍNEA
    
    return imbalance_val

In [ ]:
## Evalúa y envía tu solución
from qc_grader.challenges.spring_2022 import grade_ex3a

grade_ex3a(get_imbalance) 


Ahora, consideremos cómo la información cuántica y la entropía de entrelazamiento crece en el sistema. En un estado entrelazado, ignorar la información concerniente al resto del sistema coloca al subsistema en un estado mixto de diferentes estados. Podemos sondear la **entropía de von Neumann** para obtener información del grado de entrelazamiento entre un subsistema $A$ y el resto del sistema:

$$\mathcal{S}_{\rm vn}(\rho_A)= -\rm{tr}(\rho_A \ln \rho_A)$$

Aquí, $\rho_A= \rm{tr}_{\bar{A}} \rho$ es la matriz de densidad reducida que describe al subsistema $A$ cuando sacamos la traza con respecto al resto del sistema. Si un subsistema $A$ está completamente entrelazado con el resto del sistema entonces $\mathcal{S}_{\rm vn}(\rho_A) = \ln2$, mientras que si el subsistema es completamente separable (en un estado producto) con respecto al resto del sistema entonces $\mathcal{S}_{\rm vn}(\rho_A)=0$. Consideremos el ejemplo siguiente:

In [ ]:
bell_state = qi.Statevector(np.array([0,1,1,0])/np.sqrt(2))

rho_0 = qi.partial_trace(bell_state,[1]) # Sacamos la traza con respecto al estado 1
rho_1 = qi.partial_trace(bell_state,[0]) #Sacamos la traza con respecto al estado 0

print('QB0 vn entropy: ', qi.entropy(rho_0, base=np.exp(1)))
print('QB1 vn entropy: ', qi.entropy(rho_1, base=np.exp(1)))

## Explorando la dinámica cuántica de muchos cuerpos en una cadena de 12 qubits

Para esta parte del ejercicio, usaremos el mismo patrón del desorden de la red que usamos en el ejercicio 2.

In [ ]:
t = Parameter('t')

In [ ]:
num_qubits=12
deltas=[Parameter('delta_{:d}'.format(idx)) for idx in range(num_qubits)]
disorder_trot_step=generate_disordered_tb_instruction(t, deltas, num_qubits)

In [ ]:
# Aquí definimos el patrón del desorden

beta=(np.sqrt(5)-1)/2 # NO MODIFICAR
AA_pattern=np.cos(2*np.pi*beta*np.arange(num_qubits)) # NO MODIFICAR

<div class="alert alert-block alert-danger">
    
<b>Desafío, pregunta 3b</b> 

Prepara el sistema colocando los qubis 0, 4 y 8 en el estado  $|1\rangle$.
    
</div>

In [ ]:
delta_t=0.1
time_steps=np.arange(0,21,2)

circuits={}
Ws=[1,4,10]

for W in Ws:
    disorders=W*AA_pattern
    
    circuits[W]=[]

    for n_steps in time_steps:

        qr = QuantumRegister(num_qubits)
        qc = QuantumCircuit(qr)

        ##EDITA EL CÓDIGO DEBAJO
        
        
        ###NO MODIFICAR DEBAJO DE ESTA LÍNEA    

        for _ in range(n_steps):
            qc.append(disorder_trot_step, [i for i in range(num_qubits)])
        
        if n_steps!=0:
            qc = qc.bind_parameters({t: delta_t})
            qc = qc.bind_parameters({deltas[idx]: disorders[idx] for idx in range(num_qubits)})

        circuits[W].append(qc)

<div class="alert alert-block alert-danger">
    
<b>Desafío, pregunta 3c</b> 

Extrae la entropía de von Neumann del qubit 0 para diferentes pasos de evolución en el tiempo con diferentes valores de desorden

    
</div>

<div class="alert alert-block alert-danger">
    
<b>Desafío, pregunta 3d</b> 

Extrae el desbalance de la red para diferentes pasos de evolución en el tiempo con diferentes valores de desorden
    
</div>

In [ ]:
from qiskit import transpile

# Usa el simulador de vector de estado Aer
from qiskit import Aer

# Ejecuta el circuito cuántico en un simulador de vector de estado
backend_sim = Aer.get_backend('statevector_simulator')

probability_densities={}
state_vector_imbalances={}
vn_entropies={}

for W in tqdm(Ws):
    probability_densities[W]=[]
    state_vector_imbalances[W]=[]
    vn_entropies[W]=[]
    
    for circ in circuits[W]:

        transpiled_circ=transpile(circ, backend_sim, optimization_level=3)

        job_sim = backend_sim.run(transpiled_circ)

        # Extrae resultados
        result_sim = job_sim.result()
        outputstate = result_sim.get_statevector(transpiled_circ, decimals=6)
        ps=[]
        for idx in range(num_qubits):
            ps.append(np.abs(qi.partial_trace(outputstate,[i for i in range(num_qubits) if i!=idx]))[1,1]**2)
        
        entropy=0
        ### EDITA EL CÓDIGO DEBAJO (extrae la matriz densidad del qubit 0 calculando la traza con respecto al resto de qubits)
        
        
        ###NO MODIFICAR DEBAJO DE ESTA LÍNEA    
        
        imbalance=0
        ### EDITA EL CÓDIGO DEBAJO
        
        
        ###NO MODIFICAR DEBAJO DE ESTA LÍNEA    
        
        
        vn_entropies[W].append(entropy)
        probability_densities[W].append(ps)
        state_vector_imbalances[W].append(imbalance)

In [ ]:
fig, axs = plt.subplots(1,3,figsize=(15,5), facecolor='white', sharey=True)

for i,W in enumerate(Ws):
    ax=axs[i]
    ax.pcolormesh(np.arange(0,num_qubits,1), time_steps*delta_t ,probability_densities[W])
    ax.set_xlabel('Qubit index')
    ax.set_xticks(np.arange(1,num_qubits+1,1))

axs[0].set_ylabel('Tiempo (1/J)')

plt.show()

In [ ]:
for W in Ws:
    plt.plot(time_steps*delta_t,vn_entropies[W], '--o', label='W={:d}'.format(W))

plt.xlabel(r'Tiempo (1/J)')
plt.ylabel(r'$\mathcal{S}_{\rm vn}(\rho_0)$')
plt.legend()
plt.show()

In [ ]:
## Evalua y envía tu solución
from qc_grader.challenges.spring_2022 import grade_ex3b

grade_ex3b(vn_entropies)


In [ ]:
for W in Ws:
    plt.plot(time_steps*delta_t,state_vector_imbalances[W], '--o', label='W={:d}'.format(W))

plt.xlabel(r'Tiempo (1/J)')
plt.ylabel(r'$\mathcal{I}$')
plt.legend()
plt.show()

In [ ]:
## Evalua y envía tu solución
from qc_grader.challenges.spring_2022 import grade_ex3c

grade_ex3c(state_vector_imbalances)


# Información adicional

Traducido por: Mauricio Gómez Viloria